In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans, DBSCAN
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.decomposition import PCA


### Load Merged Data

In [ ]:
#used clean one
merged_df = pd.read_csv( '../data/interim/interim_merged_packages_receptacle_df.csv', delimiter=',')

### Sort Data Chronologically



In [ ]:

# Sort chronologically
merged_df = merged_df.sort_values('date_package').reset_index(drop=True)


### Remove unnecessary index columns that were created during the merge operation.

In [ ]:
merged_df = merged_df.drop(columns=[
    'Unnamed: 0_package',
    'Unnamed: 0_receptacle',
    #'RECPTCL_FID', 'MAILITM_FID', 'serial_number'
])



### Convert the date column to datetime format and extract temporal features:
- **hour**: Hour of day when package was processed
- **day_of_week**: Day of the week (Sunday -->Thursday)
- **is_weekend**: Binary flag indicating weekend (Friday/Saturday)

In [ ]:
merged_df['date_package'] = pd.to_datetime(merged_df['date_package'])

merged_df['hour'] = merged_df['date_package'].dt.hour
merged_df['day_of_week'] = merged_df['date_package'].dt.dayofweek
merged_df['is_weekend'] = merged_df['day_of_week'].isin([4, 5]).astype(int)



### Create a binary feature indicating whether a package exceeded the 15-day processing threshold.

In [ ]:
merged_df['delay_flag'] = (merged_df['processing_duration_days'] > 15).astype(int)



### Normalize the processing delay by the number of establishments the package passed through. This accounts for packages that take longer routes.

In [ ]:
merged_df['delay_per_etab'] = (
    merged_df['processing_duration_days'] /
    (merged_df['num_etablissements_package'] + 1)
)



### Create a route identifier by combining the current and next etablissment 

In [ ]:
merged_df['pkg_route_step'] = (
    merged_df['etablissement_postal_package'] + '→' + merged_df['next_etablissement_postal_package']
)



### Calculate how common each package route is in the dataset

In [ ]:
pkg_route_freq = merged_df['pkg_route_step'].value_counts(normalize=True)
merged_df['pkg_route_freq'] = merged_df['pkg_route_step'].map(pkg_route_freq)



### Calculate how frequently each postal establishment appears as the current or next location in the routing 

In [ ]:
etab_freq = pd.concat([
    merged_df['etablissement_postal_package'],
    merged_df['next_etablissement_postal_package']
]).value_counts()

merged_df['current_etab_freq'] = merged_df['etablissement_postal_package'].map(etab_freq)
merged_df['next_etab_freq'] = merged_df['next_etablissement_postal_package'].map(etab_freq)


### Package-Level Features


In [ ]:
pkg_features = [
    'processing_duration_days',
    'delay_flag',
    'delay_per_etab',
    'num_etablissements_package',
    'pkg_route_freq',
    'current_etab_freq',
    'next_etab_freq',
    'hour',
    'is_weekend'
]


## Receptacle Level

### receptacle route identifiers

In [ ]:
merged_df['rec_route_step'] = (
    merged_df['etablissement_postal_receptacle'] + '→' + merged_df['next_etablissement_postal_receptacle']
)




### Aggregate package-level statistics at the receptacle level to create features such as:
- Number of packages in receptacle
- Average and standard deviation of processing duration
- Average delay metrics
- Average package route rarity

In [ ]:
receptacle_route_stats = (
    merged_df
    .groupby('RECPTCL_FID')
    .agg(
        rec_route=('rec_route_step', 'first'),
        num_packages=('MAILITM_FID', 'count'),
        avg_processing_days=('processing_duration_days', 'mean'),
        std_processing_days=('processing_duration_days', 'std'),
        avg_delay_per_etab=('delay_per_etab', 'mean'),
        avg_pkg_route_rarity=('pkg_route_freq', 'mean')
    )
    .reset_index()
)

# Fill NaNs
receptacle_route_stats['std_processing_days'] = receptacle_route_stats['std_processing_days'].fillna(0)


### Receptacle Route Frequency

Calculate how common each receptacle route is

In [ ]:
rec_route_freq = receptacle_route_stats['rec_route'].value_counts(normalize=True)
receptacle_route_stats['rec_route_freq'] = receptacle_route_stats['rec_route'].map(rec_route_freq)


### Receptacle Flow Type Frequency

Map flow type frequencies to each receptacle

In [ ]:
rec_flow_type = merged_df.groupby('RECPTCL_FID')['flow_type_receptacle'].first().reset_index()

flow_type_freq = rec_flow_type['flow_type_receptacle'].value_counts(normalize=True)
rec_flow_type['flow_type_freq'] = rec_flow_type['flow_type_receptacle'].map(flow_type_freq)

receptacle_route_stats = receptacle_route_stats.merge(
    rec_flow_type[['RECPTCL_FID', 'flow_type_freq']],
    on='RECPTCL_FID',
    how='left'
)


### Receptacle Level Features

In [ ]:
rec_features = [
    'num_packages',
    'avg_processing_days',
    'std_processing_days',
    'avg_delay_per_etab',
    'avg_pkg_route_rarity',
    'rec_route_freq',
    'flow_type_freq'
]


In [ ]:
import math
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

# Check data sizes
n_packages = len(merged_df)
n_receptacles = len(receptacle_route_stats)

print(f"Package records: {n_packages}")
print(f"Receptacle records: {n_receptacles}")

# Calculate recommended max k using heuristic: k <= sqrt(n/2)
max_k_pkg = int(math.sqrt(n_packages / 2))
max_k_rec = int(math.sqrt(n_receptacles / 2))

print(f"\nMax recommended k for packages: {max_k_pkg}")
print(f"Max recommended k for receptacles: {max_k_rec}")

# Set K_range based on calculated maximums
K_range = range(2, min(max(max_k_pkg, max_k_rec) + 1, 15))

print(f"\nK_range set to: {list(K_range)}")


In [ ]:
#in pck_features find the ones with Nan
nan_pkg_features = merged_df[pkg_features].isna().sum()
nan_pkg_features = nan_pkg_features[nan_pkg_features > 0]
print("Package features with NaN values:")
print(nan_pkg_features)

In [ ]:


# Prepare data for clustering
scaler_pkg = StandardScaler()
X_pkg_scaled = scaler_pkg.fit_transform(merged_df[pkg_features])

scaler_rec = StandardScaler()
X_rec_scaled = scaler_rec.fit_transform(receptacle_route_stats[rec_features])

# Evaluate different k values for packages
silhouette_scores_pkg = []


for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_pkg_scaled)
    silhouette_scores_pkg.append(silhouette_score(X_pkg_scaled, labels))

# Evaluate different k values for receptacles
silhouette_scores_rec = []


for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_rec_scaled)
    silhouette_scores_rec.append(silhouette_score(X_rec_scaled, labels))


# Find optimal k values
best_k_pkg = list(K_range)[np.argmax(silhouette_scores_pkg)]
best_k_rec = list(K_range)[np.argmax(silhouette_scores_rec)]

print(f"\nOptimal k for packages (Silhouette): {best_k_pkg}")
print(f"Optimal k for receptacles (Silhouette): {best_k_rec}")

# Visualization - Elbow Method and Silhouette Scores
fig, axes = plt.subplots(1, 2, figsize=(15, 10))


# Package Level - Silhouette
axes[0].plot(list(K_range), silhouette_scores_pkg, 'ro-', linewidth=2, markersize=8)
axes[0].axvline(x=best_k_pkg, color='green', linestyle='--', linewidth=2, label=f'Best k={best_k_pkg}')
axes[0].set_xlabel('Number of Clusters (k)', fontsize=11)
axes[0].set_ylabel('Silhouette Score', fontsize=11)
axes[0].set_title('Silhouette Analysis - Package Level', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].legend()


# Receptacle Level - Silhouette
axes[1].plot(list(K_range), silhouette_scores_rec, 'ro-', linewidth=2, markersize=8)
axes[1].axvline(x=best_k_rec, color='green', linestyle='--', linewidth=2, label=f'Best k={best_k_rec}')
axes[1].set_xlabel('Number of Clusters (k)', fontsize=11)
axes[1].set_ylabel('Silhouette Score', fontsize=11)
axes[1].set_title('Silhouette Analysis - Receptacle Level', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].legend()

plt.tight_layout()
plt.show()

# Summary table
summary_df = pd.DataFrame({
    'k': list(K_range),
    'Silhouette_Pkg': [round(s, 4) for s in silhouette_scores_pkg],

    'Silhouette_Rec': [round(s, 4) for s in silhouette_scores_rec],
 })

print("\nCluster Quality Metrics Comparison:")
print(summary_df.to_string(index=False))


In [ ]:

# PCA for 2D visualization
pca_pkg = PCA(n_components=2)
pkg_2d = pca_pkg.fit_transform(X_pkg_scaled)

pca_rec = PCA(n_components=2)
rec_2d = pca_rec.fit_transform(X_rec_scaled)

print(f"Package PCA Explained Variance Ratio: {pca_pkg.explained_variance_ratio_}")
print(f"Cumulative Variance: {sum(pca_pkg.explained_variance_ratio_):.2%}")

print(f"\nReceptacle PCA Explained Variance Ratio: {pca_rec.explained_variance_ratio_}")
print(f"Cumulative Variance: {sum(pca_rec.explained_variance_ratio_):.2%}")

# Perform KMeans clustering on packages
optimal_k_pkg = best_k_pkg
kmeans_pkg = KMeans(n_clusters=optimal_k_pkg, random_state=42, n_init=10)
merged_df['kmeans_cluster'] = kmeans_pkg.fit_predict(X_pkg_scaled)

# Perform DBSCAN clustering on packages
dbscan_pkg = DBSCAN(eps=0.5, min_samples=5)
merged_df['dbscan_cluster'] = dbscan_pkg.fit_predict(X_pkg_scaled)

# Perform KMeans clustering on receptacles
optimal_k_rec = best_k_rec
kmeans_rec = KMeans(n_clusters=optimal_k_rec, random_state=42, n_init=10)
receptacle_route_stats['kmeans_cluster'] = kmeans_rec.fit_predict(X_rec_scaled)

# Perform DBSCAN clustering on receptacles
dbscan_rec = DBSCAN(eps=0.5, min_samples=5)
receptacle_route_stats['dbscan_cluster'] = dbscan_rec.fit_predict(X_rec_scaled)

# Visualize Package Clustering
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# KMeans package clustering
scatter1 = axes[0].scatter(pkg_2d[:, 0], pkg_2d[:, 1], c=merged_df['kmeans_cluster'], 
                           cmap='viridis', s=50, alpha=0.6, edgecolors='black', linewidth=0.5)
axes[0].set_xlabel(f'PC1 ({pca_pkg.explained_variance_ratio_[0]:.1%})', fontsize=11)
axes[0].set_ylabel(f'PC2 ({pca_pkg.explained_variance_ratio_[1]:.1%})', fontsize=11)
axes[0].set_title('Package Clustering - KMeans', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)
plt.colorbar(scatter1, ax=axes[0], label='Cluster')

# DBSCAN package clustering
scatter2 = axes[1].scatter(pkg_2d[:, 0], pkg_2d[:, 1], c=merged_df['dbscan_cluster'], 
                           cmap='plasma', s=50, alpha=0.6, edgecolors='black', linewidth=0.5)
axes[1].set_xlabel(f'PC1 ({pca_pkg.explained_variance_ratio_[0]:.1%})', fontsize=11)
axes[1].set_ylabel(f'PC2 ({pca_pkg.explained_variance_ratio_[1]:.1%})', fontsize=11)
axes[1].set_title('Package Clustering - DBSCAN', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)
plt.colorbar(scatter2, ax=axes[1], label='Cluster')

plt.tight_layout()
plt.show()

# Visualize Receptacle Clustering
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# KMeans receptacle clustering
scatter3 = axes[0].scatter(rec_2d[:, 0], rec_2d[:, 1], c=receptacle_route_stats['kmeans_cluster'], 
                           cmap='viridis', s=100, alpha=0.6, edgecolors='black', linewidth=0.5)
axes[0].set_xlabel(f'PC1 ({pca_rec.explained_variance_ratio_[0]:.1%})', fontsize=11)
axes[0].set_ylabel(f'PC2 ({pca_rec.explained_variance_ratio_[1]:.1%})', fontsize=11)
axes[0].set_title('Receptacle Clustering - KMeans', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)
plt.colorbar(scatter3, ax=axes[0], label='Cluster')

# DBSCAN receptacle clustering
scatter4 = axes[1].scatter(rec_2d[:, 0], rec_2d[:, 1], c=receptacle_route_stats['dbscan_cluster'], 
                           cmap='plasma', s=100, alpha=0.6, edgecolors='black', linewidth=0.5)
axes[1].set_xlabel(f'PC1 ({pca_rec.explained_variance_ratio_[0]:.1%})', fontsize=11)
axes[1].set_ylabel(f'PC2 ({pca_rec.explained_variance_ratio_[1]:.1%})', fontsize=11)
axes[1].set_title('Receptacle Clustering - DBSCAN', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)
plt.colorbar(scatter4, ax=axes[1], label='Cluster')

plt.tight_layout()
plt.show()


## Clustering Results Comparison

Compare KMeans and DBSCAN clustering results using 2D PCA visualization.


In [ ]:
# DBSCAN clustering for packages
# eps and min_samples need to be tuned based on your data characteristics
dbscan_pkg = DBSCAN(eps=0.5, min_samples=10)
merged_df['dbscan_cluster'] = dbscan_pkg.fit_predict(X_pkg_scaled)

# DBSCAN clustering for receptacles
dbscan_rec = DBSCAN(eps=0.5, min_samples=5)
receptacle_route_stats['dbscan_cluster'] = dbscan_rec.fit_predict(X_rec_scaled)

print(f"DBSCAN clustering completed!")
print(f"\nPackage DBSCAN clusters distribution:")
dbscan_pkg_dist = merged_df['dbscan_cluster'].value_counts().sort_index()
print(dbscan_pkg_dist)
n_clusters_pkg = len(dbscan_pkg_dist) - (1 if -1 in merged_df['dbscan_cluster'].values else 0)
n_noise_pkg = (merged_df['dbscan_cluster'] == -1).sum()
print(f"Number of clusters: {n_clusters_pkg}, Noise points: {n_noise_pkg}")

print(f"\nReceptacle DBSCAN clusters distribution:")
dbscan_rec_dist = receptacle_route_stats['dbscan_cluster'].value_counts().sort_index()
print(dbscan_rec_dist)
n_clusters_rec = len(dbscan_rec_dist) - (1 if -1 in receptacle_route_stats['dbscan_cluster'].values else 0)
n_noise_rec = (receptacle_route_stats['dbscan_cluster'] == -1).sum()
print(f"Number of clusters: {n_clusters_rec}, Noise points: {n_noise_rec}")

# Visualize DBSCAN results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

merged_df['dbscan_cluster'].value_counts().sort_index().plot(kind='bar', ax=axes[0], color='lightgreen', edgecolor='black')
axes[0].set_title('DBSCAN Cluster Distribution - Packages\n(Cluster -1 = Noise)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Cluster')
axes[0].set_ylabel('Count')
axes[0].grid(True, alpha=0.3, axis='y')

receptacle_route_stats['dbscan_cluster'].value_counts().sort_index().plot(kind='bar', ax=axes[1], color='lightyellow', edgecolor='black')
axes[1].set_title('DBSCAN Cluster Distribution - Receptacles\n(Cluster -1 = Noise)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Cluster')
axes[1].set_ylabel('Count')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


# anomaly

In [ ]:
#to do